Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на миллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
Постройте 95% доверительный интервал для средней годовой смертности в больших городах. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Будьте осторожны при использовании метода std()! Дело в том, что у объекта numpy он по умолчанию вычисляется как $1n∑i=1n(Xi−X¯)2−−−−−−−−−−−−−√$,

а у объекта pandas — как

$1n−1∑i=1n(Xi−X¯)2−−−−−−−−−−−−−−−√$.

Нас интересует только второй вариант, несмещённая оценка стандартного отклонения.

Чтобы не думать всё время о том, правильно ли вычисляется в вашем случае std(), можно всегда использовать std(ddof=1) (ddof — difference in degrees of freedom), тогда нормировка всегда будет на n-1.

In [43]:
import pandas as pd

In [44]:
df = pd.read_csv('water.txt', sep='\t')
df.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [45]:
df['town'].unique()

array(['Bath', 'Birkenhead', 'Birmingham', 'Blackburn', 'Blackpool',
       'Bolton', 'Bootle', 'Bournemouth', 'Bradford', 'Brighton',
       'Bristol', 'Burnley', 'Cardiff', 'Coventry', 'Croydon',
       'Darlington', 'Derby', 'Doncaster', 'East Ham', 'Exeter',
       'Gateshead', 'Grimsby', 'Halifax', 'Huddersfield', 'Hull',
       'Ipswich', 'Leeds', 'Leicester', 'Liverpool', 'Manchester',
       'Middlesbrough', 'Newcastle', 'Newport', 'Northampton', 'Norwich',
       'Nottingham', 'Oldham', 'Oxford', 'Plymouth', 'Portsmouth',
       'Preston', 'Reading', 'Rochdale', 'Rotherham', 'St Helens',
       'Salford', 'Sheffield', 'Southampton', 'Southend', 'Southport',
       'South Shields', 'Stockport', 'Stoke', 'Sunderland', 'Swansea',
       'Wallasey', 'Walsall', 'West Bromwich', 'West Ham',
       'Wolverhampton', 'York'], dtype=object)

In [46]:
import numpy as np
import scipy.stats

In [47]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [48]:
for i in mean_confidence_interval(df['mortality']):
    print(round(i, 4))

1524.1475
1476.0833
1572.2117


another way to do that:

In [59]:
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

_tconfint_generic(df['mortality'].mean(), 
                  df['mortality'].std(ddof=1) / np.sqrt(len(df)),
                  len(df) - 1, 0.05, 'two-sided')

(1476.0833413552848, 1572.2117406119285)

На данных из предыдущего вопроса постройте 95% доверительный интервал для средней годовой смертности по всем южным городам. Чему равна его верхняя граница? Округлите ответ до 4 знаков после десятичной точки.

In [49]:
for i in mean_confidence_interval(df.loc[df['location'] == 'South', 'mortality']):
    print(round(i, 4))

1376.8077
1320.1517
1433.4636


another way to do that:

In [61]:
_tconfint_generic(df[df.location == 'South'].mortality.mean(),
                  df[df.location == 'South'].mortality.std(ddof=1) / np.sqrt(len(df[df.location == 'South'])),
                  len(df[df.location == 'South']) - 1, 0.05, 'two-sided')

(1320.1517462936238, 1433.463638321761)

На тех же данных постройте 95% доверительный интервал для средней годовой смертности по всем северным городам. Пересекается ли этот интервал с предыдущим? Как вы думаете, какой из этого можно сделать вывод?

In [50]:
for i in mean_confidence_interval(df.loc[df['location'] == 'North', 'mortality']):
    print(round(i, 4))

1633.6
1586.5605
1680.6395


Пересекаются ли 95% доверительные интервалы для средней жёсткости воды в северных и южных городах?

In [51]:
for i in mean_confidence_interval(df.loc[df['location'] == 'South', 'hardness']):
    print(round(i, 4))
    
for i in mean_confidence_interval(df.loc[df['location'] == 'North', 'hardness']):
    print(round(i, 4))

69.7692
53.4672
86.0713
30.4
21.4225
39.3775


another way to do that:

In [63]:
_tconfint_generic(df[df.location == 'South'].hardness.mean(), 
                  df[df.location == 'South'].hardness.std(ddof=1) / np.sqrt(len(df[df.location == 'South'])), 
                  len(df[df.location == 'South']) - 1, 0.05, 'two-sided')


(53.467198692036106, 86.07126284642544)

In [64]:
_tconfint_generic(df[df.location == 'North'].hardness.mean(), 
                  df[df.location == 'North'].hardness.std(ddof=1) / np.sqrt(len(df[df.location == 'North'])), 
                  len(df[df.location == 'North']) - 1, 0.05, 'two-sided')

(21.42248728572426, 39.37751271427574)

Вспомним формулу доверительного интервала для среднего нормально распределённой случайной величины с дисперсией sigma, start superscript, 2, end superscript\sigma^2σ 
2
При sigma, equals, 1\sigma=1σ=1 какой нужен объём выборки, чтобы на уровне доверия 95% оценить среднее с точностью plus minus, 0, point, 1\pm0.1±0.1?

In [52]:
from statsmodels.stats.proportion import samplesize_confint_proportion

In [53]:
n_samples = int(np.ceil(samplesize_confint_proportion(df['mortality'].mean(), 0.05, alpha=0.05)))
n_samples

-3567181087

не сработало, поэтому взял другую выборку

In [56]:
import numpy as np
from statsmodels.stats.proportion import samplesize_confint_proportion

np.random.seed(1)
statistical_population = np.random.randint(2, size=100000)
#unique, counts = np.unique(statistical_population, return_counts=True)
#dict(zip(unique, counts))
random_sample = np.random.choice(statistical_population, size=1000)
n_samples = int(np.ceil(samplesize_confint_proportion(random_sample.mean(), 0.05, alpha=0.05)))
n_samples

385

actual way to do that:

In [68]:
from scipy import stats
np.ceil((stats.norm.ppf(1-0.05/2) / 0.1)**2)

385.0